# Network

Copyright (c) 2017 Alexander Menkin

Use of this source code is governed by an MIT-style license that can be found in the LICENSE file at
https://github.com/miloiloloo/diploma_2017_method/blob/master/LICENSE

In [ ]:
import numpy as np
import pickle
import math

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf

from keras.models import Sequential
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, UpSampling2D,core
from keras.layers import BatchNormalization, Activation, Reshape, LeakyReLU, Dropout, Flatten,Cropping2D
from keras.models import Model
from keras.datasets import mnist
from keras.optimizers import Adam
from keras.callbacks import Callback
from keras import backend as K
from keras.engine.topology import Layer, merge
from keras.engine import InputSpec
from keras import activations, initializations, regularizers, constraints,callbacks
from keras import backend as K

from sklearn.manifold import TSNE
import sklearn.metrics

from scipy import ndimage

import random
from datetime import datetime

In [ ]:
def load_learning_data(file_path):
    ''' Load data from learning data file and check it
        It returns (assemblies, probabilities)
    '''
    
    ''' Load data from learning data file '''
    try:
        f = open(file_path, 'rb')
        save = pickle.load(f)
        assemblies = save['assemblies']
        probabilities = save['probabilities']
        f.close()
        del save
    except Exception as e:
        print("Unable to load the file: " + file_path)
        raise
        
    ''' Check learning data '''
    if assemblies.shape[0] != probabilities.shape[0]:
        print("Incorrect sizes: assemblies (" + str(assemblies.shape[0]) + ") and probabilities (" + str(probabilities.shape[0]) + "). They must be equal")
        raise Exception
    
    return (assemblies, probabilities)

In [ ]:
def extract_train_valid_test_data(assemblies, probabilities, train_size, valid_size, test_size):
    ''' Separate learning data to train, valid and test data
        It returns (train_assemblies, train_probabilities, valid_assemblies, valid_probabilities, test_assemblies, test_probabilities)
    '''
    
    ''' CHECK INPUT '''
    ''' Incorrect input 1: incorrect sizes of assemblies and probabilities '''
    if assemblies.shape[0] != probabilities.shape[0]:
        raise Exception
    ''' Incorrect input 2: size of train/valid/test sets can't be negative '''
    if train_size < 0 or valid_size < 0 or test_size < 0:
        raise Exception
    ''' Incorrect input 3: incorrect sizes of train, valid and test_size'''
    if train_size + valid_size + test_size > assemblies.shape[0]:
        raise Exception
    
    ''' ALGORITHM & OUTPUT '''
    ''' Assemblies and indecies permutation '''
    permutation_indecies = np.random.permutation(assemblies.shape[0])
    assemblies = assemblies[permutation_indecies, :, :, :]
    probabilities = probabilities[permutation_indecies, :]
    
    ''' Train set '''
    train_assemblies = assemblies[0 : train_size, :, :, :]
    train_probabilities = probabilities[0 : train_size, :] 
    ''' Valid set '''
    valid_assemblies = assemblies[train_size : train_size + valid_size, :, :, :]
    valid_probabilities = probabilities[train_size : train_size + valid_size, :]
    ''' Test set '''
    test_assemblies = assemblies[train_size + valid_size : train_size + valid_size + test_size, :, :, :]
    test_probabilities = probabilities[train_size + valid_size : train_size + valid_size + test_size, :]
    
    return (train_assemblies, train_probabilities, valid_assemblies, valid_probabilities, test_assemblies, test_probabilities)

In [ ]:
def print_probabilities_stat(probabilities):
    ''' Print probabilities mean '''
    
    for class_idx in range (0, probabilities.shape[1]):
        print("class #" + str(class_idx) + ":\t" + str(np.mean(probabilities[:, class_idx])))
    
    return

In [ ]:
def get_labels_from_probabilities(probabilities):
    ''' Get labels from probabilities '''
    
    labels = np.zeros(shape=probabilities.shape)
    for idx in range(0, probabilities.shape[0]):
        labels[idx, np.argmax(probabilities[idx, :])] = 1
    return labels

In [ ]:
def normalization(train_set, valid_set, test_set):
    ''' Normalize sets '''
    
    ''' Count mean and max '''
    train_mean = np.mean(train_set)
    train_max = np.max(train_set)
    
    ''' Normalize '''
    train_set = (train_set - train_mean) / train_max
    valid_set = (valid_set - train_mean) / train_max
    test_set = (test_set - train_mean) / train_max
    
    ''' OUTPUT '''
    return (train_set, valid_set, test_set)

In [ ]:
def unite_learning_data(tuple_of_assemblies, tuple_of_probabilities):
    return (np.concatenate(tuple_of_assemblies, axis=0), np.concatenate(tuple_of_probabilities, axis=0))

def permutate_learning_data(assemblies, probabilities):
    permutation_indecies = np.random.permutation(assemblies.shape[0])
    assemblies = assemblies[permutation_indecies, :, :, :]
    probabilities = probabilities[permutation_indecies, :]
    return (assemblies, probabilities)

def get_learning_data_of_experiment(patch_size, offset, number_of_neighbours_per_side, number_of_experiment):
    array_of_assemblies = []
    array_of_probabilities = []
    # Warning: do not forget write dir path
    input_directory_path = "./learning_data/size_" + str(patch_size) + "_offset_" + str(offset) + "_left_" + str(number_of_neighbours_per_side) + "_right_" + str(number_of_neighbours_per_side) + "/"
    part_idx = 1
    try:
        while True:
            input_learning_data_file_path = input_directory_path + str(number_of_experiment) + "_" + str(part_idx) + ".pickle"
            assemblies, probabilities = load_learning_data(input_learning_data_file_path)
            print(input_learning_data_file_path)
            array_of_assemblies = array_of_assemblies + [assemblies]
            array_of_probabilities = array_of_probabilities + [probabilities]
            part_idx += 1
    except:
        print("\n")
    if len(array_of_assemblies) == 0 or len(array_of_probabilities) == 0:
        raise Exception
    return unite_learning_data(tuple(array_of_assemblies), tuple(array_of_probabilities))

In [ ]:
class mygenerator:
    def __init__(self, assemblies, probabilities, batch_size, min_stretch_k, max_stretch_k):
        
        ''' Check input '''
        if assemblies.shape[0] == 0:
            raise Exception
        if assemblies.shape[0] != probabilities.shape[0]:
            raise Exception
        if batch_size <= 0:
            raise Exception
        if min_stretch_k <= 0:
            raise Exception
        if max_stretch_k <= 0:
            raise Exception
        
        ''' Predefined parameters '''
        self._NOISE_AMPLITUDE = 0.005
        
        ''' Init '''
        self._idx = 0
        self._assemblies = assemblies
        self._probabilities = probabilities
        self._batch_size = batch_size
        self._min_stretch_k = min_stretch_k
        self._max_stretch_k = max_stretch_k
        self._permutation = np.random.permutation(self._assemblies.shape[0])
        
    def generate(self):
        
        result_assemblies = np.random.rand(
            self._batch_size,
            self._assemblies.shape[1],
            self._assemblies.shape[2],
            self._assemblies.shape[3]
        )
        result_assemblies = self._NOISE_AMPLITUDE * result_assemblies
        result_probabilities = np.ndarray(
            shape=(self._batch_size, self._probabilities.shape[1])
        )
        
        for batch_idx in range(0, self._batch_size):
            
            ''' Init probability '''
            result_probabilities[batch_idx, :] = self._probabilities[self._permutation[self._idx], :]
            
            ''' Count stretch k '''
            stretch_k_x = random.uniform(self._min_stretch_k, self._max_stretch_k)
            stretch_k_y = random.uniform(self._min_stretch_k, self._max_stretch_k)

            ''' '''
            for patch_idx in range(0, self._assemblies.shape[3]):
                
                zoom_patch = ndimage.zoom(self._assemblies[self._permutation[self._idx], :, :, patch_idx], [stretch_k_x, stretch_k_y])
                
                offset_x = (self._assemblies.shape[1] - zoom_patch.shape[0])/2
                offset_y = (self._assemblies.shape[2] - zoom_patch.shape[1])/2
                
                from_x = 0
                from_y = 0
                to_x = 0
                to_y = 0
                                
                if zoom_patch.shape[0] <= self._assemblies.shape[1]:
                    from_x = 0
                    to_x = zoom_patch.shape[0]
                else:
                    from_x = -offset_x
                    to_x = from_x + self._assemblies.shape[1]
                if zoom_patch.shape[1] <= self._assemblies.shape[2]:
                    from_y = 0
                    to_y = zoom_patch.shape[1]
                else:
                    from_y = -offset_y
                    to_y = from_y + self._assemblies.shape[2]
                
                ''' Init assembly '''
                result_assemblies[
                    batch_idx,
                    from_x + offset_x : to_x + offset_x,
                    from_y + offset_y : to_y + offset_y,
                    patch_idx
                ] = zoom_patch[
                    from_x : to_x,
                    from_y : to_y
                ]
            
            die = random.choice([0, 1])
            if die == 1:
                ''' T '''
                result_assemblies[batch_idx, :, :, :] = np.flipud(result_assemblies[batch_idx, :, :, :])
                        
            if self._idx == (self._assemblies.shape[0] - 1):
                self._permutation = np.random.permutation(self._assemblies.shape[0])
            self._idx = (self._idx + 1) % self._assemblies.shape[0]
        
        return (result_assemblies, result_probabilities)

In [ ]:
''' Load data '''
patch_size = 64
offset = 2
number_of_neighbours_per_side = 1

print("\n")
assemblies1, probabilities1 = get_learning_data_of_experiment(patch_size, offset, number_of_neighbours_per_side, 1)
assemblies2, probabilities2 = get_learning_data_of_experiment(patch_size, offset, number_of_neighbours_per_side, 2)
assemblies3, probabilities3 = get_learning_data_of_experiment(patch_size, offset, number_of_neighbours_per_side, 3)
assemblies4, probabilities4 = get_learning_data_of_experiment(patch_size, offset, number_of_neighbours_per_side, 4)


a = (assemblies1, assemblies2, assemblies3, assemblies4)
p = (probabilities1, probabilities2, probabilities3, probabilities4)
tr_a = []
tr_p = []
v_a = []
v_p = []
te_a = []
te_p = []

for i in range(0, 4):
    assemblies = a[i]
    probabilities = p[i]
    dev1 = int(0.75*assemblies.shape[0])
    dev2 = dev1 + int(0.125*assemblies.shape[0])
    
    tr_a = tr_a + [assemblies[0:dev1,:,:,:]]
    tr_p = tr_p + [probabilities[0:dev1,:]]
    
    v_a = v_a + [assemblies[dev1:dev2,:,:,:]]
    v_p = v_p + [probabilities[dev1:dev2,:]]
    
    te_a = te_a + [assemblies[dev2:assemblies.shape[0],:,:,:]]
    te_p = te_p + [probabilities[dev2:assemblies.shape[0],:]]

train_assemblies, train_probabilities = unite_learning_data(tr_a, tr_p)
valid_assemblies, valid_probabilities = unite_learning_data(v_a, v_p)
test_assemblies, test_probabilities = unite_learning_data(te_a, te_p)

permutation_indecies = np.random.permutation(train_assemblies.shape[0])
train_assemblies = train_assemblies[permutation_indecies, :, :, :]
train_probabilities = train_probabilities[permutation_indecies, :]
train_assemblies = np.clip(train_assemblies, 0, 0.1)

permutation_indecies = np.random.permutation(valid_assemblies.shape[0])
valid_assemblies = valid_assemblies[permutation_indecies, :, :, :]
valid_probabilities = valid_probabilities[permutation_indecies, :]
valid_assemblies = np.clip(valid_assemblies, 0, 0.1)

permutation_indecies = np.random.permutation(test_assemblies.shape[0])
test_assemblies = test_assemblies[permutation_indecies, :, :, :]
test_probabilities = test_probabilities[permutation_indecies, :]
test_assemblies = np.clip(test_assemblies, 0, 0.1)

train_set_size = train_assemblies.shape[0]
valid_set_size = valid_assemblies.shape[0]
test_set_size = test_assemblies.shape[0]
all_sets_size = train_set_size + valid_set_size + test_set_size


print("\nSets")
print("-------")
print("All set size:\t" + str(all_sets_size))
print("Train size:\t" + str(train_set_size))
print("Valid set size:\t" + str(valid_set_size))
print("Test set size:\t" + str(test_set_size))
print("-------\n")
print("Train probabilities stat")
print("-------")
print_probabilities_stat(train_probabilities)
print("-------\n")
print("Valid probabilities stat")
print("-------")
print_probabilities_stat(valid_probabilities)
print("-------\n")
print("Test probabilities stat")
print("-------")
print_probabilities_stat(test_probabilities)
print("-------\n")

''' Normalization '''
train_assemblies, valid_assemblies, test_assemblies = normalization(
    train_set = train_assemblies,
    valid_set = valid_assemblies,
    test_set = test_assemblies
)

''' Balancing '''
''' Get train probabilities stat '''
train_probabilities_stat = np.zeros(shape=(train_probabilities.shape[1]), dtype=train_probabilities.dtype)
for class_idx in range(0, train_probabilities.shape[1]):
    train_probabilities_stat[class_idx] = np.mean(train_probabilities[:, class_idx])
''' Balance '''
for train_idx in range(0, train_probabilities.shape[0]):
    class_idx = np.argmax(train_probabilities[train_idx, :])
    train_probabilities[train_idx, :] = np.zeros(shape=(train_probabilities.shape[1]))
    train_probabilities[train_idx, class_idx] = 1/(1 + train_probabilities_stat[class_idx])

In [ ]:
assembly_size = number_of_neighbours_per_side * 2 + 1

conv_nb_filters = None
if assembly_size < 8:    
    conv_nb_filters = [8, 16, 32, 64]
if assembly_size >= 8 and assembly_size < 16:
    conv_nb_filters = [16, 32, 48, 64]
if assembly_size >= 16 and assembly_size < 32:
    conv_nb_filters = [32, 44, 54, 64]
if assembly_size >= 32:
    raise Exception

    
input_ = Input((64, 64, assembly_size), name='input')


conv2d_1 = Convolution2D(nb_filter=conv_nb_filters[0],
                         nb_row=4,
                         nb_col=4,
                         activation='relu',
                         border_mode='same',
                         name='conv2d_1')(input_)
maxpool2d_1 = MaxPooling2D(pool_size=(2, 2), name='max_pool2d_1')(conv2d_1)
dropout_1 = Dropout(0.1, name='dropout_1')(maxpool2d_1)


conv2d_2 = Convolution2D(nb_filter=conv_nb_filters[1],
                         nb_row=4,
                         nb_col=4,
                         activation='relu',
                         border_mode='same',
                         name='conv_2d_2')(dropout_1)
maxpool2d_2 = MaxPooling2D(pool_size=(2, 2), name='max_pool_2d_2')(conv2d_2)
dropout_2 = Dropout(0.1, name='dropout_2')(maxpool2d_2)

    
conv2d_3 = Convolution2D(nb_filter=conv_nb_filters[2],
                         nb_row=4,
                         nb_col=4,
                         activation='relu',
                         border_mode='same',
                         name='conv_2d_3')(dropout_2)
maxpool2d_3 = MaxPooling2D(pool_size=(2, 2), name='max_pool_2d_3')(conv2d_3)
dropout_3 = Dropout(0.1, name='dropout_3')(maxpool2d_3)


conv2d_4 = Convolution2D(nb_filter=conv_nb_filters[3],
                         nb_row=4,
                         nb_col=4,
                         activation='relu',
                         border_mode='same',
                         name='conv_2d_4')(dropout_3)
maxpool2d_4 = MaxPooling2D(pool_size=(2, 2), name='max_pool_2d_4')(conv2d_4)


reshape_1 = Reshape((4 * 4 * 64,), name='reshape_1')(maxpool2d_4)


dense_1 = Dense(output_dim=64, activation='relu', name='dense_1')(reshape_1)


dense_2 = Dense(output_dim=5, name='dense_2')(dense_1)


activation_1 = Activation('softmax', name='activation_1')(dense_2)



model = Model(input=input_, output=activation_1)

model_for_tsne = Model(input=input_, output=dense_2)

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
model_for_tsne.compile(optimizer='adam',
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

model_for_tsne.summary()

In [ ]:
import threading

class threadsafe_iter:
    """Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def next(self):
        with self.lock:
            return self.it.next()

def threadsafe_generator(f):
    """A decorator that takes a generator function and makes it thread-safe.
    """
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))
    return g

my_generator = mygenerator(train_assemblies, train_probabilities, 32, 0.9, 1.1)

@threadsafe_generator
def generate():
    while True:
        [data,classes] = my_generator.generate()
        yield (data, classes)

In [ ]:
sess = K.get_session() 
sess.run(tf.initialize_all_variables())

In [ ]:
''' Train model '''
model.fit_generator(
    generator=generate(),
    samples_per_epoch=6400,
    nb_epoch = 10,
    validation_data=(valid_assemblies,valid_probabilities))

In [ ]:
def print_confusion_stat(probabilities, predicted_probabilities):
    
    ''' Check input '''
    if probabilities.shape[0] != predicted_probabilities.shape[0]:
        raise Exception
    if probabilities.shape[1] != predicted_probabilities.shape[1]:
        raise Exception
    
    ''' Go to classes '''
    for class_idx in range(0, probabilities.shape[1]):
        
        print("class #" + str(class_idx))
        
        TP = 0.0
        TN = 0.0
        FP = 0.0
        FN = 0.0
        for idx in range(0, probabilities.shape[0]):
            if np.argmax(predicted_probabilities[idx, :]) == class_idx:
                if np.argmax(probabilities[idx, :]) == class_idx:
                    TP += 1.0
                else:
                    FP += 1.0
            else:
                if np.argmax(probabilities[idx, :]) == class_idx:
                    FN += 1.0
                else:
                    TN += 1.0
        
        if TP + FN != 0:
            TPR = TP/(TP + FN)
        else:
            print("NO TPR, FNR, BM")
            TPR = 0
        
        if TN + FP != 0:
            TNR = TN/(TN + FP)
        else:
            print("NO TNR, FPR, BM")
            TNR = 0
        
        if TP + FP != 0:
            PPV = TP/(TP + FP)
        else:
            print("NO PPV, FDR, MK")
            PPV = 0
        
        if TN + FN != 0:
            NPV = TN/(TN + FN)
        else:
            print("NO NPV, FOR, MK")
            NPV = 0
        
        FNR = 1 - TPR
        FPR = 1 - TNR
        FDR = 1 - PPV
        FOR = 1 - NPV
        ACC = (TP + TN)/(TP + FN + TN + FP)
        F_1 = (2*TP)/(2*TP + FP + FN)
        
        if TP + FP != 0 and TP + FN != 0 and TN + FP != 0 and TN + FN != 0:
            MCC = (TP*TN - FP*FN)/(math.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN)))
        else:
            print("NO MCC") 
            MCC = 0
        
        BM = TPR + TNR - 1
        MK = PPV + NPV - 1
        
        print("-------")
        print("TP:\t" + str(TP))
        print("TN:\t" + str(TN))
        print("FP:\t" + str(FP))
        print("FN:\t" + str(FN))
        print("TPR:\t" + str(TPR))
        print("TNR:\t" + str(TNR))
        print("PPV:\t" + str(PPV))
        print("NPV:\t" + str(NPV))
        print("FNR:\t" + str(FNR))
        print("FPR:\t" + str(FPR))
        print("FDR:\t" + str(FDR))
        print("FOR:\t" + str(FOR))
        print("ACC:\t" + str(ACC))
        print("F_1:\t" + str(F_1))
        print("MCC:\t" + str(MCC))
        print("BM:\t" + str(BM))
        print("MK:\t" + str(MK))
        print("-------\n")
    
    return

In [ ]:
def show_stat_table(probabilities, predicted_probabilities):
    
    ''' Check input '''
    if probabilities.shape[0] != predicted_probabilities.shape[0]:
        raise Exception
    if probabilities.shape[1] != predicted_probabilities.shape[1]:
        raise Exception
        
    class_stat_table = np.zeros(shape=(probabilities.shape[1], probabilities.shape[1]), dtype=np.float32)
    for idx in range(0, probabilities.shape[0]):
        first_idx = np.argmax(probabilities[idx, :])
        second_idx = np.argmax(predicted_probabilities[idx, :])
        class_stat_table[first_idx, second_idx] += 1
    for first_idx in range(0, probabilities.shape[1]):
        class_stat_table[first_idx, :] = class_stat_table[first_idx, :] / np.sum(class_stat_table[first_idx, :])
    return class_stat_table

In [ ]:
''' Test model '''
test_score = model.evaluate(test_assemblies, test_probabilities, batch_size=16)

''' Print model's test stat '''
print("\n\nTest model stat")
print("-------")
print("Loss:\t\t" + str(test_score[0]))
print("Accuracy:\t" + str(test_score[1]))
print("-------\n")

test_predicted_probabilities = model.predict(test_assemblies)

print_confusion_stat(
    probabilities=test_probabilities,
    predicted_probabilities = test_predicted_probabilities
)

cst = show_stat_table(
    probabilities=test_probabilities,
    predicted_probabilities = test_predicted_probabilities
)

In [ ]:
import itertools
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Purples):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

plt.figure()
plot_confusion_matrix(np.round(cst,2), classes=['UNDEF', 'LOCOM', 'IMMOB', 'REAR', 'GROOM'],
                      title='Confusion matrix, without normalization')

plt.show()

In [ ]:
RS = 0
tsne = TSNE(n_components=2, n_iter=1000, random_state=RS, metric='euclidean')
X_t = tsne.fit_transform(model_for_tsne.predict(test_assemblies))
y = np.zeros(shape=(test_probabilities.shape[0]))
for ind in range(0, test_probabilities.shape[0]):
    y[ind] = np.argmax(test_probabilities[ind,:])

In [ ]:
plt.figure()
#plt.scatter(X_t[np.where(y == 0), 0],X_t[np.where(y == 0), 1],marker='o', color='r',linewidth='1', label='undef')  
plt.scatter(X_t[np.where(y == 1), 0],X_t[np.where(y == 1), 1],marker='o', color='g',linewidth='0.05', label='locom')
plt.scatter(X_t[np.where(y == 2), 0],X_t[np.where(y == 2), 1],marker='o', color='b',linewidth='0.05', label='immob')
plt.scatter(X_t[np.where(y == 3), 0],X_t[np.where(y == 3), 1],marker='o', color='c',linewidth='0.05', label='rear')
plt.scatter(X_t[np.where(y == 4), 0],X_t[np.where(y == 4), 1],marker='o', color='m',linewidth='0.05', label='groom')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()